In [ ]:
import pandas as pd
DATA_PATH = "C:\Users\kangq\OneDrive - Johns Hopkins\Documents\ToolRL\dataset\rlla_4k\train.parquet"

# Load the dataset into a pandas DataFrame
df = pd.read_parquet(DATA_PATH)

# Basic information about the loaded DataFrame
print("Dataset loaded from:", DATA_PATH)
print("Number of rows, columns:", df.shape)

: 

In [ ]:
print("Columns in the dataset:\n")
print(df.dtypes)

In [ ]:
df.head()
   pip install --no-build-isolation --no-deps \